In [10]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
# Define custom InceptionV3 model
def custom_inceptionv3(input_shape=(224, 224, 3), num_classes=5):
    input_tensor = layers.Input(shape=input_shape)

    # Stem
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), padding='valid', activation='relu')(input_tensor)
    x = layers.Conv2D(32, (3, 3), padding='valid', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = layers.Conv2D(80, (1, 1), padding='valid', activation='relu')(x)
    x = layers.Conv2D(192, (3, 3), padding='valid', activation='relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    # Inception blocks
    x = inception_block(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_block(x, filters=[128, 128, 192, 32, 96, 64])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = inception_block(x, filters=[192, 96, 208, 16, 48, 64])
    x = inception_block(x, filters=[160, 112, 224, 24, 64, 64])
    x = inception_block(x, filters=[128, 128, 256, 24, 64, 64])
    x = inception_block(x, filters=[112, 144, 288, 32, 64, 64])
    x = inception_block(x, filters=[256, 160, 320, 32, 128, 128])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = inception_block(x, filters=[256, 160, 320, 32, 128, 128])
    x = inception_block(x, filters=[384, 192, 384, 48, 128, 128])

    # Global average pooling and output layer
    x = layers.GlobalAveragePooling2D()(x)
    # x = layers.Dropout(0.4)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = Model(input_tensor, output, name='custom_inceptionv3')
    return model

# Define Inception block
def inception_block(x, filters):
    branch1x1 = layers.Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    branch3x3 = layers.Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    branch3x3 = layers.Conv2D(filters[2], (3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = layers.Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    branch5x5 = layers.Conv2D(filters[4], (5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = layers.Conv2D(filters[5], (1, 1), padding='same', activation='relu')(branch_pool)

    return layers.concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)

In [12]:
# Instantiate the custom InceptionV3 model
model = custom_inceptionv3()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model architecture
# model.summary()

# Paths to your train and test directories
train_dir = '/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/train'
test_dir = '/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/test'

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 200 images belonging to 5 classes.
Found 88 images belonging to 5 classes.


In [13]:
# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator))

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.2061 - loss: 1.6130 - val_accuracy: 0.2045 - val_loss: 1.6087
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100


2024-03-14 03:22:24.094443: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-14 03:22:24.121704: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.1716 - loss: 1.6100 - val_accuracy: 0.2045 - val_loss: 1.6091
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100


2024-03-14 03:22:41.460022: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-14 03:22:41.479893: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.1921 - loss: 1.6089 - val_accuracy: 0.2045 - val_loss: 1.6094
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100


...

2024-03-14 03:37:12.347024: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-14 03:37:12.356227: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.1993 - loss: 1.5990 - val_accuracy: 0.2045 - val_loss: 1.6088
Epoch 98/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 99/100


2024-03-14 03:37:33.799101: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-14 03:37:33.811031: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.2123 - loss: 1.6056 - val_accuracy: 0.2045 - val_loss: 1.6105
Epoch 100/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


2024-03-14 03:37:54.913509: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-14 03:37:54.924263: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [14]:
# Extract training and validation accuracy from history
training_accuracy = (history.history['accuracy'][-1] * 100)
validation_accuracy = (history.history['val_accuracy'][-1] * 100)

print(f"Training Accuracy: {training_accuracy}")
print(f"Validation Accuracy: {validation_accuracy}")

Training Accuracy: 0.0
Validation Accuracy: 0.0


In [15]:
import numpy as np
from keras.preprocessing import image

# Load the image
img_path = '/Users/vishnuvardhanvankayalapati/Desktop/FinalProject_Bird_Image_Classification/Project_Data_Set/train/Red_faced_Cormorant/Red_Faced_Cormorant_0001_796327.jpg'
img = image.load_img(img_path, target_size=(img_height, img_width))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize the image

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class
predicted_class = np.argmax(predictions[0])

# If you have ground truth labels, compare with the ground truth to compute accuracy
ground_truth =[0,1,2,3,4]# Provide the ground truth label here
accuracy = (predicted_class == ground_truth)

# Print the predicted class and probability
print("Predicted Class:", predicted_class)
print("Predicted Probability:", predictions[0][predicted_class])

# If you have ground truth labels, print accuracy
print("Prediction Accuracy:", accuracy)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
Predicted Class: 0
Predicted Probability: 0.21167429
Prediction Accuracy: [ True False False False False]
